# Recopilación de datos de temperatura de una API

## Sobre los datos
En este notebook, recogeremos datos diarios de temperatura de la API [de los Centros Nacionales de Información Medioambiental (NCEI)](https://www.ncdc.noaa.gov/cdo-web/webservices/v2). Utilizaremos el conjunto de datos Global Historical Climatology Network - Daily (GHCND); consulte la documentación [aquí](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf).

*Nota: El NCEI forma parte de la Administración Nacional Oceánica y Atmosférica (NOAA) y, como se puede ver en la URL de la API, este recurso se creó cuando el NCEI se llamaba NCDC. Si la URL de este recurso cambiara en el futuro, puede buscar "NCEI weather API" para encontrar la actualizada.*

## Uso de la API del NCEI
Solicite su token [aquí](https://www.ncdc.noaa.gov/cdo-web/token) y péguelo a continuación.

In [ ]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()
NOAA_TOKEN = os.getenv('NOAA_TOKEN')

def make_request(endpoint:str, payload=None) -> requests.Response:
    """
    Realizar una solicitud a un punto final específico de la API meteorológica
    pasando las cabeceras y la carga útil opcional.
    
    Parámetros:
        - endpoint: El punto final de la API a la que
                    realizar una solicitud GET.
        - payload: Un diccionario de datos para pasar junto
                   con la solicitud.
    
    Devuelve:
        Un objeto de respuesta.
    """
    return requests.get(
        f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',
        headers={
            'token': NOAA_TOKEN
        },
        params=payload
    )

**Nota: la API nos limita a 5 solicitudes por segundo y 10.000 solicitudes al día.

## Ver qué conjuntos de datos están disponibles
Podemos realizar peticiones al punto final `datasets` para ver qué conjuntos de datos están disponibles. También pasamos un diccionario para la carga útil para obtener conjuntos de datos que tengan datos posteriores a la fecha de inicio del 1 de octubre de 2018.

In [ ]:
response = make_request('datasets', {'startdate': '2018-10-01'})

El código de estado `200` significa que todo va bien. Puede encontrar más códigos [aquí](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).

In [ ]:
response.status_code

Alternativamente, podemos comprobar el atributo `ok`:

In [ ]:
response.ok

### Obtener las claves del resultado
El resultado es una carga JSON, a la que podemos acceder con el método `json()` de nuestro objeto respuesta. Los objetos JSON pueden ser tratados como diccionarios, por lo que podemos acceder a las claves como lo haríamos con un diccionario:

In [ ]:
payload = response.json()
payload.keys()

Los metadatos de la respuesta nos darán información sobre la solicitud y los datos que recibimos de vuelta:

In [ ]:
payload['metadata']

### Averiguar qué datos hay en el resultado
La clave `results` contiene los datos que hemos solicitado. Se trata de una lista de lo que serían las filas de nuestro marco de datos. Cada entrada de la lista es un diccionario, por lo que podemos consultar las claves para obtener los campos:

In [ ]:
payload['results'][0].keys()

### Parsear el resultado
No queremos todos esos campos, así que usaremos una comphrensión de lista para sacar sólo los campos `id` y `name`:

In [ ]:
[(data['id'], data['name']) for data in payload['results']]

## Averiguar qué categoría de datos queremos
Los datos `GHCND` que contienen resúmenes diarios son los que queremos. Ahora tenemos que hacer otra petición para averiguar qué categorías de datos queremos recoger. Este es el endpoint `datacategories`. Tenemos que pasar el `datasetid` de `GHCND` como carga útil para que la API sepa sobre qué conjunto de datos estamos preguntando:

In [ ]:
# obtener id de categoría de datos
response = make_request(
    'datacategories', payload={'datasetid': 'GHCND'}
)
response.status_code

Como sabemos que la API nos da una clave `metadata` y otra `results` en cada respuesta, podemos ver qué hay en la parte `results` de la carga JSON:

In [ ]:
response.json()['results']

## Obtener el tipo de datos ID para la categoría de temperatura
Vamos a trabajar con temperaturas, así que queremos la categoría de datos `TEMP`. Ahora, tenemos que encontrar los `datatypes` a recoger. Para ello, utilizamos el endpoint `datatypes` y proporcionamos el `datacategoryid`, que es `TEMP`. También especificamos un límite para el número de `datatypes` a devolver con la carga. Si hay más que esto podemos hacer otra solicitud más tarde, pero por ahora, sólo queremos elegir unos pocos:

In [ ]:
# obtener id de tipo de datos
response = make_request(
    'datatypes',
    payload={
        'datacategoryid': 'TEMP', 
        'limit': 100
    }
)
response.status_code

Podemos obtener los campos `id` y `name` de cada una de las entradas de la parte de datos `results`. Los campos que nos interesan están en la parte inferior:

In [ ]:
[(datatype['id'], datatype['name']) for datatype in response.json()['results']][-5:] # mira los últimos 5

## Determinar qué categoría de ubicación queremos
Ahora que sabemos qué `datatypes` vamos a recoger, tenemos que encontrar la ubicación a utilizar. En primer lugar, tenemos que averiguar la categoría de ubicación. Esto se obtiene del endpoint `locationcategories` pasando el `datasetid`:

In [ ]:
# obtener id de categoría de ubicación
response = make_request(
    'locationcategories', 
    payload={'datasetid': 'GHCND'}
)
response.status_code

Podemos utilizar `pprint` para imprimir los diccionarios en un formato más fácil de leer. Después de hacerlo, podemos ver que hay 12 categorías de ubicación diferentes, pero sólo nos interesa `CITY`:

In [ ]:
import pprint
pprint.pprint(response.json())

## Obtener el ID de localización de NYC
Para encontrar el ID de ubicación de Nueva York, tenemos que buscar en todas las ciudades disponibles. Como podemos pedir a la API que nos devuelva las ciudades ordenadas, podemos usar la búsqueda binaria para encontrar Nueva York rápidamente sin tener que hacer muchas peticiones o solicitar muchos datos a la vez. La siguiente función realiza la primera petición para ver el tamaño de la lista y busca el primer valor. A partir de ahí decide si tiene que moverse hacia el principio o el final de la lista comparando el elemento que buscamos con los demás alfabéticamente. Cada vez que hace una petición puede descartar la mitad de los datos que quedan por buscar.

In [ ]:
def get_item(name:str, what:dict, endpoint:str, start=1, end=None) -> dict:
    """
    Obtiene la carga útil JSON de un campo dado por su nombre utilizando la búsqueda binaria.

    Parámetros:
        - name: El elemento a buscar.
        - what: Diccionario que especifica cuál es el elemento en `name`.
        - endpoint: Dónde buscar el elemento.
        - inicio: La posición en la que empezar. No necesitamos tocar esto, pero la función
                 manipulará esto con la recursión.
        - fin: La última posición de los elementos. Se usa para encontrar el punto medio, pero
               como `start` no es algo de lo que tengamos que preocuparnos.

    Devuelve:
        Diccionario con la información del ítem si se encuentra de lo contrario
        un diccionario vacío.
    """
    # encontrar el punto medio que utilizamos para cortar los datos por la mitad cada vez
    mid = (start + (end or 1)) // 2
    
    # escribe el nombre en minúsculas para que no se distinga entre mayúsculas y minúsculas
    name = name.lower()
    
    # define el payload que enviaremos con cada petición
    payload = {
        'datasetid': 'GHCND',
        'sortfield': 'name',
        'offset': mid, # cambiaremos el offset cada vez
        'limit': 1 # sólo queremos un valor de vuelta
    }
    
    # realiza nuestra petición añadiendo cualquier parámetro de filtro adicional de `what`.
    response = make_request(endpoint, {**payload, **what})
    
    if response.ok:
        payload = response.json()

        # si la respuesta es correcta, tomar el índice final de los metadatos de respuesta la primera vez
        end = end or payload['metadata']['resultset']['count']
        
        # toma la versión en minúsculas del nombre actual
        current_name = payload['results'][0]['name'].lower()
        
        # si lo que buscamos está en el nombre actual, hemos encontrado nuestro artículo
        if name in current_name:
            return payload['results'][0] # devolver el elemento encontrado
        else:
            if start >= end: 
                # si nuestro índice de inicio es mayor o igual que nuestro final, no pudimos encontrarlo
                return {}
            elif name < current_name:
                # nuestro nombre viene antes del nombre actual en el alfabeto, así que buscamos más a la izquierda
                return get_item(name, what, endpoint, start, mid - 1)
            elif name > current_name:
                # nuestro nombre viene después del nombre actual en el alfabeto, así que buscamos más a la derecha
                return get_item(name, what, endpoint, mid + 1, end)    
    else:
        # la respuesta no era correcta, usa el código para determinar por qué
        print(f'Respuesta no OK, estado: {response.status_code}')

Cuando utilizamos la búsqueda binaria para encontrar Nueva York, la encontramos en sólo 8 peticiones a pesar de estar cerca de la mitad de las 1.983 entradas:

In [ ]:
# obtener id NYC
nyc = get_item('New York', {'locationcategoryid': 'CITY'}, 'locations')
nyc

## Obtener el ID de estación para Central Park
Los datos más detallados se encuentran a nivel de estación:

In [ ]:
central_park = get_item('NY City Central Park', {'locationid': nyc['id']}, 'stations')
central_park

## Solicitar los datos de temperatura
Por fin tenemos todo lo que necesitamos para solicitar los datos de temperatura de Nueva York. Para ello, utilizamos el endpoint `data` y proporcionamos todos los parámetros que hemos ido recogiendo a lo largo de nuestra exploración de la API:

In [ ]:
# obtenga los datos de los resúmenes diarios de NYC 
response = make_request(
    'data', 
    {
        'datasetid': 'GHCND',
        'stationid': central_park['id'],
        'locationid': nyc['id'],
        'startdate': '2018-10-01',
        'enddate': '2018-10-31',
        'datatypeid': ['TAVG', 'TMAX', 'TMIN'], # temperatura media, máxima y mínima
        'units': 'metric',
        'limit': 1000
    }
)
response.status_code

## Crear un DataFrame
La estación de Central Park sólo tiene las temperaturas mínimas y máximas diarias.

In [ ]:
import pandas as pd

df = pd.DataFrame(response.json()['results'])
df.head()

No conseguimos `TAVG` porque la estación no mide eso:

In [ ]:
df.datatype.unique()

A pesar de aparecer en los datos como medición... ¡Los datos del mundo real están sucios!

In [ ]:
if get_item(
    'NY City Central Park', {'locationid': nyc['id'], 'datatypeid': 'TAVG'}, 'stations'
):
    print('Found!')

## Usando una estación diferente
Usemos en su lugar el aeropuerto de LaGuardia. Contiene `TAVG` (temperatura media diaria):

In [ ]:
laguardia = get_item(
    'LaGuardia', {'locationid': nyc['id']}, 'stations'
)
laguardia

Esta vez hacemos nuestra solicitud utilizando la estación del aeropuerto de LaGuardia.

In [ ]:
# obtenga los datos de los resúmenes diarios de NYC
response = make_request(
    'data', 
    {
        'datasetid': 'GHCND',
        'stationid': laguardia['id'],
        'locationid': nyc['id'],
        'startdate': '2018-10-01',
        'enddate': '2018-10-31',
        'datatypeid': ['TAVG', 'TMAX', 'TMIN'], # temperatura en el momento de la observación, mín. y máx.
        'units': 'metric',
        'limit': 1000
    }
)
response.status_code

La solicitud se ha realizado correctamente, así que vamos a crear un marco de datos:

In [ ]:
df = pd.DataFrame(response.json()['results'])
df.head()

Debemos comprobar que hemos obtenido lo que queríamos: 31 entradas para TAVG, TMAX y TMIN (1 por día):

In [ ]:
df.datatype.value_counts()

Escribe los datos en un archivo CSV para utilizarlos en otros cuadernos.

In [ ]:
df.to_csv('data/nyc_temperatures.csv', index=False)

<hr>
<div>
    <a href="./1-ancho_vs_largo.ipynb">
        <button>&#8592; Notebook Anterior</button>
    </a>
    <a href="./3-cleaning_data.ipynb">
        <button style="float: right;"> Siguiente Notebook &#8594;</button>
    </a>
</div>
<hr>